In [2]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
import matplotlib.pyplot as plt

%matplotlib inline

## 1. 准备一些预处理的参数

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
x_train.shape

(50000, 32, 32, 3)

In [5]:
a = np.random.randn(4,5,5,3)

In [11]:
np.mean(a, axis=(0,1,2))

array([ 0.02135011, -0.02332792, -0.10546028])

In [12]:
# 以channels来划分，计算均值、标准差
np.mean(x_train, axis=(0,1,2))

array([ 125.30691805,  122.95039414,  113.86538318])

In [13]:
np.std(x_train, axis=(0,1,2))

array([ 62.99321928,  62.08870764,  66.70489964])

In [14]:
def color_preprocessing(x_train, x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_mean = [125.307,  122.950,  113.865]
    x_std = [62.993,  62.089,  66.705]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - x_mean[i]) / x_std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - x_mean[i]) / x_std[i]
    return x_train, x_test

## 2. 模型搭建

In [15]:
weight_decay = 0.0001

In [20]:
def build_model():
    model = Sequential()
    
    model.add(Conv2D(192, (5,5), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay), input_shape=(32,32,3))) 
    model.add(Conv2D(160, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) 
    model.add(Conv2D(96, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(192, (5,5), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(192, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(192, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) 
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(192, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(192, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(10, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) 
    
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
    
    
    

## 3. 数据处理和模型训练

In [17]:
x_train, x_test = color_preprocessing(x_train, x_test)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [18]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

In [25]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [26]:
train_x.shape, valid_x.shape, train_y.shape, valid_y.shape

((45000, 32, 32, 3), (5000, 32, 32, 3), (45000, 10), (5000, 10))

In [21]:
model = build_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 192)       14592     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 160)       30880     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 96)        15456     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 192)       460992    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 192)       37056     
__________

In [22]:
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.125, height_shift_range=0.125, 
                             fill_mode='constant', cval=0.)

In [27]:
from time import time

t0 = time()
datagen.fit(train_x)
print("Time : ", time()-t0)

Time :  0.27900004386901855


In [ ]:
# 1500 ~ 45000/32=1407
history = model.fit_generator(datagen.flow(train_x, train_y, batch_size=32), steps_per_epoch=1500, epochs=50, 
                              validation_data=datagen.flow(valid_x, valid_y, batch_size=32), validation_steps=157)
model.save('cnn_cifar10.h5')

Epoch 1/50
1500/1500 [==============================] - 1301s 867ms/step - loss: 2.3093 - acc: 0.1009 - val_loss: 2.3032 - val_acc: 0.1018
Epoch 2/50
 800/1500 [===============>..............] - ETA: 9:47 - loss: 2.3030 - acc: 0.0994

In [ ]:
# 1500 ~ 45000/32=1407
history = model.fit_generator(datagen.flow(train_x, train_y, batch_size=32), steps_per_epoch=1500, epochs=50, 
                              validation_data=ImageDataGenerator().flow(valid_x, valid_y, batch_size=32), validation_steps=157)
model.save('cnn_cifar10.h5')